# 作業 : 實作英文-德文翻譯機器人
***
## [作業目標]
用 PyTorch 實作一個英文-德文翻譯機器人

## [作業重點]
* 語言資料處理
* 使用 LSTM 建構 Encoder: EncoderLSTM
* 使用 LSTM 建構 Decoder: DecoderLSTM
* 搭建 Sequence to Sequence 模型: Seq2Seq
* 撰寫訓練函式
* 撰寫測試函式

## [問題]
在 Colab 上實際執行完這個範例後，請改用 BiLSTM 來建構 Encoder

## 引用需要的模組

In [1]:
import os
import spacy
import torch
import random
import pandas as pd
from pprint import pprint

from torch import nn, optim
from torchtext.data import Field, BucketIterator
from torchtext.datasets import Multi30k
from torchtext.data.metrics import bleu_score
from torch.utils.tensorboard import SummaryWriter

## 下載 spacy 英文/德文語料

In [2]:
!python -m spacy download en
spacy_en = spacy.load('en')
!python -m spacy download de
spacy_de = spacy.load('de')

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 14.9MB 8.3MB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp36-none-any.whl size=14907057 sha256=38563853eab721191ad9a5599d0a9fca687a728d255b9fa4e16e984724d0be9e
  Stored in directory: /tmp/pip-ephem-wheel-cache-6x1r_5tq/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [3]:
def tokenize_en(text):
    return [token.text for token in spacy_en.tokenizer(text)]

def tokenize_de(text):
    return [token.text for token in spacy_de.tokenizer(text)]

# Sample Run
sample_text = 'I love machine learning'
print(tokenize_en(sample_text))

english = Field(tokenize=tokenize_en, lower=True,
                init_token='<sos>', eos_token='<eos>')
german = Field(tokenize=tokenize_de, lower=True,
               init_token='<sos>', eos_token='<eos>')

train_data, valid_data, test_data = Multi30k.splits(exts=('.en', '.de'), fields=(english, german))
english.build_vocab(train_data, max_size=10000, min_freq=2)
german.build_vocab(train_data, max_size=10000, min_freq=2)

print(f"Unique tokens in source (english) vocabulary: {len(english.vocab)}")
print(f"Unique tokens in target (german) vocabulary: {len(german.vocab)}")

['I', 'love', 'machine', 'learning']
downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:02<00:00, 560kB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 174kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 166kB/s]


Unique tokens in source (english) vocabulary: 5893
Unique tokens in target (german) vocabulary: 7855


In [4]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

print(train_data[5].__dict__.keys())
pprint(train_data[5].__dict__.values())

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000
dict_keys(['src', 'trg'])
dict_values([['a', 'man', 'in', 'green', 'holds', 'a', 'guitar', 'while', 'the', 'other', 'man', 'observes', 'his', 'shirt', '.'], ['ein', 'mann', 'in', 'grün', 'hält', 'eine', 'gitarre', ',', 'während', 'der', 'andere', 'mann', 'sein', 'hemd', 'ansieht', '.']])


In [5]:
BATCH_SIZE = 32
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE, 
    sort_within_batch=True,
    sort_key=lambda x: len(x.src),
    device=device
)

In [6]:
len_eng_examples = []
len_ger_examples = []
for idx, data in enumerate(train_data):
    len_eng_examples.append(len(data.src))
    len_ger_examples.append(len(data.trg))
    if idx < 10:
        print('German - ', *data.src, ' Length - ', len(data.src))
        print('English - ', *data.trg, ' Length - ', len(data.trg))
        print()

print(f"Maximum Length of English sentence {max(len_eng_examples)} and German sentence {max(len_ger_examples)} in the dataset")
print(f"Minimum Length of English sentence {min(len_eng_examples)} and German sentence {min(len_ger_examples)} in the dataset")

German -  two young , white males are outside near many bushes .  Length -  11
English -  zwei junge weiße männer sind im freien in der nähe vieler büsche .  Length -  13

German -  several men in hard hats are operating a giant pulley system .  Length -  12
English -  mehrere männer mit schutzhelmen bedienen ein antriebsradsystem .  Length -  8

German -  a little girl climbing into a wooden playhouse .  Length -  9
English -  ein kleines mädchen klettert in ein spielhaus aus holz .  Length -  10

German -  a man in a blue shirt is standing on a ladder cleaning a window .  Length -  15
English -  ein mann in einem blauen hemd steht auf einer leiter und putzt ein fenster .  Length -  15

German -  two men are at the stove preparing food .  Length -  9
English -  zwei männer stehen am herd und bereiten essen zu .  Length -  10

German -  a man in green holds a guitar while the other man observes his shirt .  Length -  15
English -  ein mann in grün hält eine gitarre , während der andere

In [7]:
data = next(iter(train_iterator))
temp_eng, temp_ger = data.src, data.trg
print('Shapes', data.src.shape, data.trg.shape, '\n')
print('English - ', *data.src, ' Length - ', len(data.src), '\n')
print('German - ', *data.trg, ' Length - ', len(data.trg))

Shapes torch.Size([13, 32]) torch.Size([16, 32]) 

English -  tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0') tensor([  38,   16,    4,   16,    4,   21,    4,    4,    4, 4125,    7,    4,
          48,    4,    4,    4,    4,    4,    7,    4,    4,    4,    4,    4,
           4,  324,   16,    4,    4,    4,    4,    4], device='cuda:0') tensor([  12,   50,    9,   30,  602,  145,    9,    9,   25,  252, 2296,  903,
          30,   64,   35, 1814, 3504,  168,  552,  153,    9,   53,   64,   14,
          53,  327,   19,  648,   33,   33,   34,   14], device='cuda:0') tensor([ 168,   78,   10,   17, 1435,    9,    6,   22,   35,    4,   15,  212,
          17,   10,   13, 1543,  648,   42,   10,   10,  940,   33,  195,  321,
          34,   14,  152,  831,    6,    6,   22,   22], device='cuda:0') tensor([ 42,   6, 119,  78, 540,  32,  25, 561,  10, 606, 145, 171, 252,  78,
          4,  15, 972, 433, 

In [8]:
temp_eng_idx = temp_eng.cpu().detach().numpy()
temp_ger_idx = temp_ger.cpu().detach().numpy()

In [9]:
df_eng_idx = pd.DataFrame(data=temp_eng_idx,
                          columns=[str('S_') + str(x + 1) for x in range(BATCH_SIZE)])
df_eng_idx.index.name = 'Time Steps'
df_eng_idx.index = df_eng_idx.index + 1 
df_eng_idx

,S_1,S_2,S_3,S_4,S_5,S_6,S_7,S_8,S_9,S_10,S_11,S_12,S_13,S_14,S_15,S_16,S_17,S_18,S_19,S_20,S_21,S_22,S_23,S_24,S_25,S_26,S_27,S_28,S_29,S_30,S_31,S_32
Time Steps,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
2,38,16,4,16,4,21,4,4,4,4125,7,4,48,4,4,4,4,4,7,4,4,4,4,4,4,324,16,4,4,4,4,4
3,12,50,9,30,602,145,9,9,25,252,2296,903,30,64,35,1814,3504,168,552,153,9,53,64,14,53,327,19,648,33,33,34,14
4,168,78,10,17,1435,9,6,22,35,4,15,212,17,10,13,1543,648,42,10,10,940,33,195,321,34,14,152,831,6,6,22,22
5,42,6,119,78,540,32,25,561,10,606,145,171,252,78,4,15,972,433,635,6,4,10,4,82,10,6,6,60,4,4,4,4
6,1320,4,8,8,18,8,89,525,1608,13,9,7,4,4,409,12,7,9,6,4,55,41,576,14,22,29,1030,7,26,796,1355,29
7,30,295,27,4,82,4,6,10,1678,826,1884,399,1318,198,15,251,2853,1225,65,4365,8,4,18,102,4,23,1545,103,23,372,67,23
8,395,244,286,295,28,293,43,1812,798,11,734,2765,93,40,256,15,8,6,78,37,4,35,557,4986,3676,107,171,28,37,56,137,2623
9,5802,17,6,40,8,45,12,4,4,131,6,8,4,4,6,1812,4,4,4,13,223,49,4,4,6,40,4,169,4,20,6,8


In [10]:
idx2word = {idx: word for idx, word in enumerate(english.vocab.itos)}
df_eng_word = pd.DataFrame(columns=[str('S_') + str(x + 1) for x in range(BATCH_SIZE)])
df_eng_word = df_eng_idx.replace(idx2word)
df_eng_word

,S_1,S_2,S_3,S_4,S_5,S_6,S_7,S_8,S_9,S_10,S_11,S_12,S_13,S_14,S_15,S_16,S_17,S_18,S_19,S_20,S_21,S_22,S_23,S_24,S_25,S_26,S_27,S_28,S_29,S_30,S_31,S_32
Time Steps,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>
2,group,two,a,two,a,an,a,a,a,indians,the,a,three,a,a,a,a,a,the,a,a,a,a,a,a,african,two,a,a,a,a,a
3,of,women,man,men,military,old,man,man,white,having,fat,athlete,men,person,dog,rod,lonely,middle,surfer,baby,man,little,person,woman,little,american,people,skier,girl,girl,boy,woman
4,middle,riding,is,are,member,man,in,wearing,dog,a,",",runs,are,is,with,crew,skier,-,is,is,pushes,girl,using,painting,boy,woman,walk,flies,in,in,wearing,wearing
5,-,in,talking,riding,talks,sitting,white,safety,is,gathering,old,across,having,riding,a,",",enjoys,aged,coming,in,a,is,a,another,is,in,in,through,a,a,a,a
6,eastern,a,on,on,to,on,stands,goggles,airborne,with,man,the,a,a,frisbee,of,the,man,in,a,child,walking,ladder,woman,wearing,blue,opposite,the,black,cat,birthday,blue
7,men,motorcycle,his,a,another,a,in,is,whilst,coats,falls,racing,discussion,horse,",",five,slopes,crouches,from,sandpit,on,a,to,'s,a,shirt,directions,air,shirt,costume,hat,shirt
8,selling,they,cellphone,motorcycle,while,bus,front,repairing,catching,and,asleep,lanes,behind,down,swimming,",",on,in,riding,playing,a,dog,paint,fingernails,snorkel,looks,across,while,playing,looking,plays,typing
9,trinkets,are,in,down,on,holding,of,a,a,food,in,on,a,a,in,repairing,a,a,a,with,toy,by,a,a,in,down,a,doing,a,at,in,on


## 用 LSTM 搭建 Encoder 類別: EncoderLSTM

In [11]:
class EncoderLSTM(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, drop_rate):
        super(EncoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, 
                            bidirectional=True, dropout=drop_rate)
        self.dropout = nn.Dropout(drop_rate)

    def forward(self, x):
        embedding = self.dropout(self.embedding(x))
        outputs, (hidden_state, cell_state) = self.LSTM(embedding)
        hidden_state = hidden_state.view(self.num_layers, -1, self.hidden_size * 2)
        cell_state = cell_state.view(self.num_layers, -1, self.hidden_size * 2)

        return hidden_state, cell_state

input_size_encoder = len(english.vocab)
encoder_embedding_size = 300
hidden_size = 256
num_layers = 2
encoder_dropout = 0.5

encoder_lstm = EncoderLSTM(input_size_encoder, encoder_embedding_size,
                           hidden_size, num_layers, encoder_dropout).to(device)
print(encoder_lstm)

EncoderLSTM(
  (embedding): Embedding(5893, 300)
  (LSTM): LSTM(300, 256, num_layers=2, dropout=0.5, bidirectional=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


## 用 LSTM 搭建 decoder 類別: DecoderLSTM

In [12]:
class DecoderLSTM(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size,
                 num_layers, output_size, drop_rate):
        super(DecoderLSTM, self).__init__()
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.LSTM = nn.LSTM(embedding_size, hidden_size,
                            num_layers, dropout=drop_rate)
        self.fc = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(drop_rate)

    def forward(self, x, hidden_state, cell_state):
        x = x.unsqueeze(0)
        embedding = self.dropout(self.embedding(x))
        outputs, (hidden_state, cell_state) = self.LSTM(embedding, (hidden_state, cell_state))
        predictions = self.fc(outputs)
        predictions = predictions.squeeze(0)

        return predictions, hidden_state, cell_state

input_size_decoder = len(german.vocab)
decoder_embedding_size = 300
hidden_size = 512
num_layers = 2
decoder_dropout = 0.5
output_size = len(german.vocab)

decoder_lstm = DecoderLSTM(input_size_decoder, decoder_embedding_size,
                           hidden_size, num_layers, output_size, decoder_dropout).to(device)
print(decoder_lstm)

DecoderLSTM(
  (embedding): Embedding(7855, 300)
  (LSTM): LSTM(300, 512, num_layers=2, dropout=0.5)
  (fc): Linear(in_features=512, out_features=7855, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [13]:
batch = next(iter(train_iterator))
print(batch.src.shape)
print(batch.trg.shape)

x = batch.trg[1]
print(x)

torch.Size([12, 32])
torch.Size([14, 32])
tensor([221,   5,  17,   8, 644,   8,   5,   5,  18,   8,   5,   8,   8,   8,
          5,   5,   5,   8,   5,  76,   5,   5,   5,   5,   5,  18,   5,   8,
         18,  18,   5,  18], device='cuda:0')


## Seq2Seq(Sequence to Sequence)類別

In [14]:
class Seq2Seq(nn.Module):
    def __init__(self, Encoder_LSTM, Decoder_LSTM):
        super(Seq2Seq, self).__init__()
        self.Encoder_LSTM = Encoder_LSTM
        self.Decoder_LSTM = Decoder_LSTM

    def forward(self, source, target, tfr=0.5):
        batch_size = source.shape[1]
        target_len = target.shape[0]
        target_vocab_size = len(german.vocab)
        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)

        hidden_state, cell_state = self.Encoder_LSTM(source)

        x = target[0]   # Trigger token <SOS>
        for i in range(1, target_len):
            output, hidden_state, cell_state = self.Decoder_LSTM(x, hidden_state, cell_state)
            outputs[i] = output
            best_guess = output.argmax(1)   # 0th dimension is batch size, 1st dimension is word embedding size
            # Either pass the next word correctly from the dataset or use the earlier predicted word
            x = target[i] if random.random() < tfr else best_guess

        return outputs

In [15]:
# Hyperparameters
learning_rate = 0.001
step = 0
writer = SummaryWriter(os.path.join('runs', 'loss_plot'))

model = Seq2Seq(encoder_lstm, decoder_lstm).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

pad_idx = english.vocab.stoi['<pad>']
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

In [16]:
model

Seq2Seq(
  (Encoder_LSTM): EncoderLSTM(
    (embedding): Embedding(5893, 300)
    (LSTM): LSTM(300, 256, num_layers=2, dropout=0.5, bidirectional=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (Decoder_LSTM): DecoderLSTM(
    (embedding): Embedding(7855, 300)
    (LSTM): LSTM(300, 512, num_layers=2, dropout=0.5)
    (fc): Linear(in_features=512, out_features=7855, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [17]:
def translate(model, sentence, english, german, device, max_length=50):
    if type(sentence) == str:
        tokens = tokenize_en(sentence)
    else:
        tokens = [token.lower() for token in sentence]
    tokens.insert(0, english.init_token)
    tokens.append(english.eos_token)
    text_to_indices = [english.vocab.stoi[token] for token in tokens]
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    # Build encoder hidden, cell state
    with torch.no_grad():
        hidden, cell = model.Encoder_LSTM(sentence_tensor)

    outputs = [german.vocab.stoi['<sos>']]
    for _ in range(max_length):
        previous_word = torch.LongTensor([outputs[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.Decoder_LSTM(previous_word, hidden, cell)
            best_guess = output.argmax(1).item()

        outputs.append(best_guess)

        # Model predicts it's the end of the sentence
        if output.argmax(1).item() == german.vocab.stoi['<eos>']:
            break

    translated_sentence = [german.vocab.itos[idx] for idx in outputs]
    
    return translated_sentence[1:]   # ignore <sos> token

In [18]:
def checkpoint_and_save(model, best_loss, epoch, optimizer, epoch_loss):
    print('saving\n')
    state = {'model': model, 'best_loss': best_loss, 'epoch': epoch,
             'rng_state': torch.get_rng_state(), 'optimizer': optimizer.state_dict()}
    torch.save(state, 'checkpoint-NMT')
    torch.save(model.state_dict(), 'checkpoint-NMT-SD')

In [19]:
# 用來評估模型的函式: bleu
def bleu(data, model, english, german, device):
    targets, outputs = [], []
    for example in data:
        src = vars(example)["src"]
        trg = vars(example)["trg"]

        prediction = translate(model, src, english, german, device)
        prediction = prediction[:-1]  # remove <eos> token

        targets.append([trg])
        outputs.append(prediction)

    return bleu_score(outputs, targets)

In [20]:
epoch_loss = 0.0
num_epochs = 30
best_loss = 999999
best_epoch = -1
sentence = 'a man in a blue shirt is standing on a ladder and cleaning a window'
ts = []

for epoch in range(num_epochs):
    model.eval()
    print(f"Epoch - {epoch + 1} / {num_epochs}")
    translated_sentence = translate(model, sentence, english, german, device, max_length=50)
    print(f"Translated example sentence: \n {translated_sentence}")
    ts.append(translated_sentence)

    model.train()
    for batch_idx, batch in enumerate(train_iterator):
        input = batch.src.to(device)
        target = batch.trg.to(device)

        # Pass the input and target for model's forward method
        output = model(input, target)
        output = output[1:].view(-1, output.shape[2])
        target = target[1:].view(-1)

        # Calculate the loss value for every epoch
        loss = criterion(output, target)
        
        # Clear the accumulating gradients
        optimizer.zero_grad()

        # Calculate the gradients for weights & biases using back-propagation
        loss.backward()

        # Clip the gradient value is it exceeds > 5.0
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)

        # Update the weights values using the gradients we calculated using bp 
        optimizer.step()
        step += 1
        epoch_loss += loss.item()
        writer.add_scalar('Training loss', loss, global_step=step)

    if epoch_loss < best_loss:
        best_epoch = epoch
        best_loss = epoch_loss
        checkpoint_and_save(model, best_loss, epoch, optimizer, epoch_loss) 
        if (epoch - best_epoch >= 10):
            print('no improvement in 10 epochs, break')
            break
    print(f"Epoch_Loss - {loss.item()}\n")

print(epoch_loss / len(train_iterator))

Epoch - 1 / 30
Translated example sentence: 
 ['verzieht', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd', 'polohemd']
saving

Epoch_Loss - 4.2687249183654785

Epoch - 2 / 30
Translated example sentence: 
 ['ein', 'mann', 'in', 'einem', 'einem', 'hemd', 'und', 'einem', 'einem', '<unk>', '.', '<eos>']
Epoch_Loss - 4.260138988494873

Epoch - 3 / 30
Translated example sentence: 
 ['ein', 'mann', 'in', 'einem', 'blauen', 'oberteil', 'und', 'einem', 'einem', '<unk>', '.', 